# 準備

In [1]:
from dotenv import load_dotenv
load_dotenv() # .envファイルから環境変数を読み込み(OPENAI_API_KEY変数にAPI Keyが入っている)

True

# LLM Chain

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # これだけでモデルを初期化できるぽい

In [3]:
# これでモデルと会話することができる
# print(llm.invoke("langsmithはどのようにテストを支援してくれますか？"))

In [4]:
# プロンプトテンプレートを使うこともできる
from langchain_core.prompts import ChatPromptTemplate

# この例だとシステムに「技術ドキュメントライター」というロールを与えている
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは世界レベルの技術ドキュメントライターです。"),
    ("user", "{input}")
])

In [5]:
# 組み合わせてchainをつくる
chain = prompt | llm

In [6]:
# chainもinvokeで会話できる
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

In [7]:
# 出力をAIMessageからStringに変える
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
# chainに加える
chain = prompt | llm | output_parser

In [9]:
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

## Diving Deeper

さらに深く理解するためにたくさんのドキュメントが用意されている。今すぐにはとても読めないので、少しずつ読む。

# Retrieval Chain

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview") # URLを渡すとその内容を取ってきてくれる

docs = loader.load() # 指定したURLからドキュメントを取得する

In [11]:
from langchain_openai import OpenAIEmbeddings # ベクターストアに格納するためにドキュメントをベクトル化するためのモジュール

embeddings = OpenAIEmbeddings()

# これを使って、今からベクターストアにドキュメントを挿入する

In [12]:
# 簡単のため、シンプルなローカルベクターストアであるFAISSを使う
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter() # チャンクサイズの制限を下回るまで再帰的にテキストを分割するTextSplitter
documents = text_splitter.split_documents(docs) # TextSpritterに書けてテキストを分割する
vector = FAISS.from_documents(documents, embeddings) # 分割した文書をベクトル化してベクターストアに保存する

In [13]:
documents

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain 

# 与えたコンテキストに基づいて質問に回答させるためのプロンプト
prompt = ChatPromptTemplate.from_template("""与えられた文脈のみに基づいて質問に回答してください:
<context>
{context}
</context>

質問: {input}""")

# 質問と取得したドキュメントを受け取り、解答を生成するチェーンを作成する
# create_stuff_documents_chainはドキュメントのリストをコンテキストとしてプロンプトに渡す
document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document 

# ドキュメントを自分で渡すことで、チェーンを自分で実行することもできる
# document_chain.invoke({
#     "input": "how can langsmith help with testing?",
#     "context": [Document(page_content="langsmith can let you visualize test results")]
# })

In [16]:
# でも、retrieverを使うことで、質問に最も関連性の高いドキュメントを動的に選択してモデルに渡すことができる
from langchain.chains import create_retrieval_chain # retrieval chainを作るための関数

retriever = vector.as_retriever() # retrieverを作成
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retrieverとドキュメントを受け取るchainからretrieval chainを作成

In [17]:
# これで、retrieval chainを呼び出すことができる
# response = retrieval_chain.invoke({"input": "LangSmithはどのようにテストを支援してくれますか?"})
# print(response["answer"])

# LangSmith offers several features that can help with testing:...

## Diving Deeper

これも少しずつ読む

# Conversation Retrieval Chain

ここまでやってきたのは1つの質問に答えるchain。でもこれだと一問一答で会話ができない。フォローアップクエスチョンにもこたえられるchainを作るにはどうすればいいだろうか。

In [18]:
# create_retrieval_chain()をまだ使うことができる。でも変更を加える必要がある
# 1. 直近の入力だけでなく、すべての会話の歴史を入力に加える
# 2. 最後のLLM chainも同様にすべての会話の歴史を受け取って出力を生成する

from langchain.chains import create_history_aware_retriever # 歴史を考慮したretriever
from langchain_core.prompts import MessagesPlaceholder # メッセージプレースホルダー (何なのかよくわからないぞ) 

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"), # メッセージプレースホルダの中にchat_historyを入れる
    ("user", "{input}"), # ユーザのインプット
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    # 以上を踏まえて、会話に関係する情報を取得するために検索する検索クエリを生成してください
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt) # 歴史を踏まえた生成を行うchainを生成

In [19]:
from langchain_core.messages import HumanMessage, AIMessage # 人間のメッセージとAIのメッセージ

# 仮の歴史
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか？"), AIMessage(content="もちろんです!")]

# chainに入力して続きを出力
# retriever_chain.invoke({
#     "chat_history": chat_history,
#     "input": "どのように支援してくれるのですか"
# })

# LLMはクエリを生成するため、LangSmithを使ったテストに関するドキュメントを返す(なんでクエリを生成したらドキュメントが返ってくるのかわかんない)

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "以下のコンテキストを参照してユーザーの質問に答えてください:\n\n{context}"), # コンテキスト
    MessagesPlaceholder(variable_name="chat_history"), # 会話の歴史
    ("user", "{input}"), # ユーザーの入力
])
document_chain = create_stuff_documents_chain(llm, prompt) # 変更したプロンプトで再びchainを生成

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain) # 歴史も参照できるchainを生成

In [21]:
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか?"), AIMessage(content="Yes!")]
# retrieval_chain.invoke({
#     "chat_history": chat_history,
#     "input": "どのように支援してくれますか？"
# })

# Agent

LLMがどのような手順を実行するのかを決定する

In [22]:
# まずはエージェントがどのツールにアクセスできるのかを定義する
from langchain.tools.retriever import create_retriever_tool # 今までのはcreate_retrieval_chainだった

retriever_tool = create_retriever_tool(
    retriever, # 会話の履歴を取ってくるもの
    "langsmith_search", # ツールの名前?
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!", # このツールの使い方をモデルに説明してる
    # ↑ LangSmithに関する情報を探せ。LangSmithに関する質問が来たら、このツールを使わなくてはなりませんよ！
)

In [23]:
# 検索ツールとしてTavilyを使う
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults() # エージェントが使う検索ツール

In [24]:
# エージェントが使うツールのリスト
tools = [retriever_tool, search]

In [25]:
from langchain_openai import ChatOpenAI # LLMモデルをロードするためのモジュール
from langchain import hub # プロンプトをpullするためのモジュール
from langchain.agents import create_openai_functions_agent # エージェントを作るためのモジュール
from langchain.agents import AgentExecutor # エージェントを作るためのモジュール

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent") # プロンプトをpullしてくる
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) # モデルを指定してAPIをロード
agent = create_openai_functions_agent(llm, tools, prompt) # モデル、ツール、プロンプトをまとめて渡し、エージェントを初期化
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # エージェントを実行する準備を完了

In [26]:
# agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [27]:
# agent_executor.invoke({"input": "what is the weather in SF?"})

In [28]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")

In [29]:
# [
#     {
#         "content": "LangSmithはLLMアプリケーションのテストを支援してくれますか？",
#         "type": "human"
#     },
#     {
#         "content": "もちろんです！",
#         "type": "ai"
#     }
# ]

In [32]:
chat_history = [{"content": "LangSmithはLLMアプリケーションのテストを支援してくれますか？", "type": "human"}, {"content": "もちろんです！", "type": "ai"}]
remote_chain.invoke({"input": "LangSmithって何ですか", "chat_history": chat_history})

HTTPStatusError: Server error '500 Internal Server Error' for url 'http://localhost:8000/agent/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500 for Internal Server Error

In [36]:
agent_executor.invoke({"input": "langsmithって何ですか？", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}), Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and a

KeyboardInterrupt: 